## Задание: 
### Дан граф, в котором каждой дуге приписано два значения: буква в строке на входе и звук на выходе (во втором случае может стоять символ ожидания, означающий, что выдачу необходимо проводить по окончании последовательности символов: .-[a; ə]->.; .-[s; *]->.-[h; ʃ ]->.). Написать программу, осуществляющую поиск последовательностей фонем в тексте на входе.

### Фонетические правила, по которым мы будем играть в рамках этого ноутбука

Существующие фонемы: 
- гласные: а, о, у, ы, э, и
- твердые согласные: б, в, г, д, з, ж, л, м, р, н, п, ф, к, т, с, ш, х, ц, щ, ч
- мягкие согласные: б', в', г', д', з', п', ф', к', т', с', х', щ, ч, л', м', р', н', j

Как можно заметить уже здесь, мы не помечаем апострофом мягкость тех согласных фонем, которые и так как правило мягкие: j, ч, щ.

Приведенные фонемы можно разделить на следующие группы: 
- сонорные: л, м, р, н, л', м', р', н', j
- шумные звонкие: б, в, г, д, з, ж, б', в', г', д', з'
- шумные глухие: п, ф, к, т, с, ш, х, ц, щ, ч, п', ф', к', т', с', х', щ, ч

Если для каждой пары согласных фонем (твердый - мягкий) существует одна графема (например, б и б' выражаются на письме как б), то с гласными ситуация чуть другая: 
- графемы, обозначающие дифтонгоид j + гласный: я, е, ё, ю
- графемы, которые этого не делают: а, о, у, ы, э, и
- графемы, обозначающие мягкость предыдущего согласного: я, е, ё, ю, и
- графемы, которые этого не делают: а, о, у, ы, э,

In [31]:
plain_vowels = ['а', 'о', 'у', 'ы', 'э', 'и']
j_vowels = ['я', 'е', 'ё', 'ю']

'''монофтонгические соответствия для каждого гл'''
to_plain = {'я': 'а', 'е': 'э', 'ё': 'о',
 'ю': 'у', 'а': 'а', 'о': 'о', 'у': 'у',
 'ы': 'ы', 'э': 'э', 'и': 'и'}

non_front_vowels = ['а', 'о', 'у', 'ы', 'э']
front_vowels = ['я', 'е', 'ё', 'ю','и']

sonant_cons = ['л', 'м', 'р', 'н']
voiced_cons = ['б', 'в', 'г', 'д','з', 'ж']
unvoiced_cons = ['п', 'ф', 'к', 'т','с', 'ш', 'х', 'ц', 'щ', 'ч']

'''глухие соответствия для каждого согл'''
devoiced = {'б': 'п', 'в': 'ф', 'г': 'к',
 'д': 'т', 'з': 'с', 'ж': 'ш', 'л': 'л',
 'м': 'м', 'р': 'р', 'н': 'н', 'п': 'п',
 'ф': 'ф', 'к': 'к', 'т': 'т', 'с': 'с',
 'ш': 'ш', 'х': 'х', 'ц': 'ц', 'щ': 'щ',
 'ч': 'ч'}

'''мягкие соответствия для каждого согл'''
soft = {'б': "б'", 'в': "в'", 'г': "г'",
 'д': "д'", 'з': "з'", 'ж': 'ж',
 'п': "п'", 'ф': "ф'", 'к': "к'",
 'т': "т'", 'с': "с'", 'ш': 'ш',
 'х': "х'", 'ц': 'ц','щ': 'щ', 'ч': 'ч',
 'л': "л'", 'м': "м'", 'р': "р'", 'н': "н'"}

'''звонкие соответствия для каждого согл'''
vocalized = {'п': 'б', 'ф': 'в', 'к': 'г', 'т': 'д',
 'с': 'з', 'ш': 'ж', 'х': 'х', 'ц': 'ц', 'щ': 'щ',
 'ч': 'ч', 'б': 'б', 'в': 'в', 'г': 'г', 'д': 'д',
 'з': 'з', 'ж': 'ж', 'л': 'л', 'м': 'м', 'р': 'р', 'н': 'н'}

Сонорный реализуется как
- сонорный твердый, если после него идет
    - а, о, у, ы, э
    - звонкий согласный
    - глухой согласный
    - сонорный
    - конец слова
- сонорный мягкий, если после него идет
    - ь
    - ъ
    - и, е, я, ю, е, ё
    - j
    

Глухой реализуется как
- твердый глухой, если после него идет
    - глухой согласный
    - а, о, у, ы, э
    - сонорный
    - в
    - конец слова
- мягкий глухой, если после него идет
    - я, ю, е, и, ё
    - ь$
    - ь + глухой
    - ъ
    - j
- твердый звонкий, если после него идет
    - звонкий (не в)
- мягкий звонкий, если после него идет
    - ь + звонкий (не в)


Звонкий реализуется как
- звонкий твердый, если после него идет
    - звонкий
    - сонорный
    - а у о ы э
- звонкий мягкий, если после него 
    - я, ю, е, ё, и
    - ь, а потом
        - звонкий
        - гласный
        - сонорный
- глухой твердый, если после него
    - глухой
    - $
- глухой мягкий, если после него 
    - глухой
    - конец слова
        

In [32]:
def start(char, previous):
    if char in plain_vowels:
        return start, char, char
    if char in j_vowels:
        return start, 'j' + to_plain[char], char
    if char in voiced_cons:
        return voiced, '', char
    if char in unvoiced_cons:
        return unvoiced, '', char
    if char in sonant_cons:
        return sonant, '', char
    if char == 'й':
        return start, 'j', char
    if char == '$':
        return stop, '', char

def sonant(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons:
        return voiced, previous, char
    if char in unvoiced_cons:
        return unvoiced, previous, char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, previous, char

def voiced(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons:
        return voiced, previous, char
    if char in unvoiced_cons:
        return unvoiced, devoiced[previous], char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, devoiced[previous], char

def unvoiced(char, previous):
    if char in non_front_vowels:
        return start, previous + char, char
    if char in front_vowels:
        return start, soft[previous] + to_plain[char], char
    if char in voiced_cons and not char == 'в':
        return voiced, vocalized[previous], char
    if char in unvoiced_cons or char == 'в':
        return unvoiced, previous, char
    if char in sonant_cons:
        return sonant, previous, char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == 'ь':
        return soft_sign, '', previous
    if char == 'ъ':
        return hard_sign, '', previous
    if char == '$':
        return stop, previous, char

def soft_sign(char, previous):
    if char in non_front_vowels or char in front_vowels:
        return start, soft[previous] + 'j' + to_plain[char], char
    if char in voiced_cons and not char == 'в':
        return voiced, soft[vocalized[previous]], char
    if char in unvoiced_cons or char == 'в':
        return unvoiced, soft[devoiced[previous]], char
    if char in sonant_cons or char == 'в':
        return sonant, soft[previous], char
    if char == 'й':
        return start, soft[previous] + 'j', char
    if char == '$':
        return stop, soft[devoiced[previous]], char
    pass

def hard_sign(char, previous):
    if char in non_front_vowels or char in front_vowels:
        return start, soft[previous] + 'j' + to_plain[char], char
    
    
def stop(char):
    pass

In [33]:
def transcribe(string):
    state = start
    previous = ''
    output = ''
    try:
        for symbol in string:
            state, answer, previous = state(symbol, previous)
            output += answer
    except TypeError:
        return False, string # bad input
    return True, output

In [50]:
transcribe('интернационал$')

(True, "инт'эрнационал")

In [51]:
transcribe('ъуъ$')

(False, 'ъуъ$')

In [9]:
def preprocessing(text):
    tokens = []
    for word in text.split():
        token = []
        for symbol in word:
            if symbol.isalpha():
                token.append(symbol)
        if len(token) > 0:
            tokens.append(''.join(token).lower() + '$')
    return tokens

In [10]:
def transcribe_text(text):
    tokens = preprocessing(text)
    transcribed = []
    bad_input = []
    for word in tokens:
        approve, transcription = transcribe(word)
        if approve:
            transcribed.append(transcription)
        else:
            bad_input.append(transcription)
    phonematic_text = ' '.join(transcribed)
    return phonematic_text, bad_input

In [59]:
with open('Балашовская правда корпус/1001.txt', encoding='utf-8') as file:
    contents = file.read().split('=====')[-1]

print(contents + '\n\n\n')
text, bad = transcribe_text(contents)
print(text)
print('bad: ', bad)

В режиме видеоконференции прошел зональный родительский совет  для пяти переговорных площадок области. Проблемы взаимодействия семьи и школы обсудили 260 делегатов родительской общественности из 47 муниципальных районов Саратовской области.  Главной целью мероприятия стало разъяснение целей и задач модернизации региональной системы образования,  получение обратной связи от родительской общественности и выявление проблемных зон в школах области. 
 
На площадке, организованной в Балашове на  базе гимназии им. Ю.А. Гарнаева, собрались родители Аркадакского, Балашовского, Екатериновского, Калининского, Лысогорского, Романовского, Ртищевского, Самойловского и Турковского районов.  Видеоконференцию открыла кандидат педагогических наук, ректор ГАОУ ДПО «СарИПКиПРО» И.М. Ильковская.  
В ходе видеоконференции представителям родительской общественности была предоставлена возможность  задать в чате вопрос заместителю министра образования Саратовской области Н.Н. Обрежа. Родителей интересовали воп

In [14]:
def phonematic_search(sequence, text):
    transcription, bad_inputs = transcribe_text(text)
    if len(bad_inputs):
        line = ', '.join(bad_inputs)
        line = f'Unable to transcribe the following:{line}'
        print(line)
    return sequence in transcription

In [20]:
with open('Балашовская правда корпус/1000.txt', encoding='utf-8') as file:
    contents = file.read().split('=====')[-1]
    
    
print('Текст статьи:\n', contents)
sequence = "чэлов'эк"
print(phonematic_search(sequence, contents))
sequence = "мар'ина"
print(phonematic_search(sequence, contents))

Текст статьи:
 В зале заседаний районной администрации не было свободных мест. На совещание, посвященное изменениям в пенсионном законодательстве, были приглашены специалисты кадровых служб, бухгалтеры предприятий и учреждений. 
 
Заместитель начальника управления Пенсионного фонда России в Балашовском районе Г.Н. Стоякина проинформировала собравшихся о том, что в 2014 году пенсии будут повышать дважды – с 1 февраля и с 1 апреля. Трудовые пенсии по старости возрастут на 8,1 %, социальные – на 17, 6 %. Изменились требования к предоставлению сведений о выходящих на пенсию. Списки граждан, у которых подходит срок назначения пенсии, необходимо предоставить в управление ПФ за девять месяцев до наступления пенсионного возраста. 
Сведения о выходящих на пенсию работниках обязан отправить в электронном виде в управление ПФ специалист кадровой службы или бухгалтер, назначенный приказом работодателя ответственным за передачу данных в Пенсионный фонд. Работник может самостоятельно собрать и сдать